In [2]:
"""
AQI Data Cleaning - OPTIMIZED FOR ML ACCURACY
==============================================
Improvements:
- Removes 100% NaN columns (snwd, wpgt, tsun)
- Better feature engineering
- More data preservation
- Optimized for model performance
"""

import pandas as pd
import numpy as np
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("AQI DATA CLEANING - OPTIMIZED VERSION")
print("="*70)

INPUT_FILE = "data/cleaned_aqi_data_v2.csv"
OUTPUT_FILE = "data/cleaned_aqi_data_v2.csv"
REPORT_FILE = "data/data_quality_report_v2.txt"

# ============================================================================
# STEP 1: Load Data
# ============================================================================

def load_data(filepath):
    """Load raw AQI data"""
    print(f"\nStep 1: Loading data from {filepath}")
    
    try:
        df = pd.read_csv(filepath)
        print(f"✓ Loaded {len(df)} rows, {len(df.columns)} columns")
        return df
    except Exception as e:
        print(f"✗ ERROR: {e}")
        return None

# ============================================================================
# STEP 2: Remove Useless Columns
# ============================================================================

def remove_useless_columns(df):
    """Remove columns that are 100% NaN or have no predictive value"""
    print("\nStep 2: Removing useless columns...")
    
    initial_cols = len(df.columns)
    
    # Remove columns with 100% missing values
    nan_cols = df.columns[df.isnull().all()].tolist()
    if nan_cols:
        df = df.drop(columns=nan_cols)
        print(f"✓ Removed {len(nan_cols)} columns with 100% NaN: {nan_cols}")
    
    # Remove columns with >95% missing values
    high_nan_cols = df.columns[df.isnull().sum() / len(df) > 0.95].tolist()
    if high_nan_cols:
        df = df.drop(columns=high_nan_cols)
        print(f"✓ Removed {len(high_nan_cols)} columns with >95% NaN: {high_nan_cols}")
    
    # Remove non-predictive text columns (keep only useful categorical)
    text_cols_to_remove = ['aqi_color', 'time_of_day']  # These are redundant
    existing_to_remove = [col for col in text_cols_to_remove if col in df.columns]
    if existing_to_remove:
        df = df.drop(columns=existing_to_remove)
        print(f"✓ Removed redundant columns: {existing_to_remove}")
    
    final_cols = len(df.columns)
    print(f"✓ Columns: {initial_cols} → {final_cols} (removed {initial_cols - final_cols})")
    
    return df

# ============================================================================
# STEP 3: Smart Missing Value Handling
# ============================================================================

def clean_missing_values_smart(df):
    """Handle missing values intelligently"""
    print("\nStep 3: Handling missing values (smart strategy)...")
    
    initial_rows = len(df)
    
    # For pollutants: forward fill (pollution persists)
    pollutant_cols = ['pm2_5', 'pm10', 'nitrogen_dioxide', 'ozone']
    for col in pollutant_cols:
        if col in df.columns:
            before = df[col].isnull().sum()
            if before > 0:
                # Forward fill with limit to avoid unrealistic propagation
                df[col] = df[col].fillna(method='ffill', limit=3)
                df[col] = df[col].fillna(method='bfill', limit=3)
                # Fill remaining with median (more robust than mean)
                df[col] = df[col].fillna(df[col].median())
                after = df[col].isnull().sum()
                print(f"  {col}: {before} → {after} nulls")
    
    # For weather: interpolate (smooth transitions)
    weather_cols = ['temp', 'rhum', 'prcp', 'wdir', 'wspd', 'pres', 'cldc', 'coco']
    for col in weather_cols:
        if col in df.columns:
            before = df[col].isnull().sum()
            if before > 0:
                df[col] = df[col].interpolate(method='linear', limit_direction='both')
                # Fill remaining with median
                df[col] = df[col].fillna(df[col].median())
                after = df[col].isnull().sum()
                if before != after:
                    print(f"  {col}: {before} → {after} nulls")
    
    # For AQI: forward fill (recent values are good predictors)
    aqi_cols = ['aqi', 'aqi_pm25', 'aqi_pm10', 'aqi_o3', 'aqi_no2']
    for col in aqi_cols:
        if col in df.columns:
            before = df[col].isnull().sum()
            if before > 0:
                df[col] = df[col].fillna(method='ffill', limit=6)
                df[col] = df[col].fillna(df[col].median())
                after = df[col].isnull().sum()
                if before != after:
                    print(f"  {col}: {before} → {after} nulls")
    
    # Drop rows only if critical columns are still missing
    critical_cols = ['time', 'aqi']
    df = df.dropna(subset=[col for col in critical_cols if col in df.columns])
    
    final_rows = len(df)
    print(f"✓ Rows: {initial_rows} → {final_rows} (lost {initial_rows - final_rows})")
    
    return df

# ============================================================================
# STEP 4: Handle Outliers Carefully
# ============================================================================

def handle_outliers_smart(df):
    """Handle outliers without losing too much data"""
    print("\nStep 4: Handling outliers...")
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    # Don't cap AQI-related columns (they can spike legitimately)
    exclude_cols = ['aqi', 'aqi_pm25', 'aqi_pm10', 'aqi_o3', 'aqi_no2', 
                    'hour', 'day_of_week', 'month', 'is_weekend', 'day_of_month']
    
    for col in numeric_cols:
        if col not in exclude_cols:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            
            # Use wider bounds (5 IQR instead of 3) to preserve more data
            lower_bound = Q1 - 5 * IQR
            upper_bound = Q3 + 5 * IQR
            
            outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
            
            if outliers > 0:
                df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
                print(f"  {col}: Capped {outliers} extreme outliers")
    
    print(f"✓ Outliers handled")
    return df

# ============================================================================
# STEP 5: Fix Data Types
# ============================================================================

def fix_data_types(df):
    """Ensure correct data types"""
    print("\nStep 5: Fixing data types...")
    
    if 'time' in df.columns:
        df['time'] = pd.to_datetime(df['time'])
        print("  ✓ time → datetime")
    
    # Numeric columns
    numeric_cols = ['pm2_5', 'pm10', 'nitrogen_dioxide', 'ozone', 
                    'temp', 'rhum', 'prcp', 'wdir', 'wspd', 'pres', 'cldc', 'coco']
    
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Integer columns
    int_cols = ['hour', 'day_of_week', 'day_of_month', 'month', 'is_weekend',
                'aqi', 'aqi_pm25', 'aqi_pm10', 'aqi_o3', 'aqi_no2']
    
    for col in int_cols:
        if col in df.columns:
            df[col] = df[col].fillna(0).astype(int)
    
    print("✓ Data types fixed")
    return df

# ============================================================================
# STEP 6: MINIMAL Feature Engineering (Prevent Overfitting)
# ============================================================================

def engineer_features_minimal(df):
    """Create ONLY essential features"""
    print("\nStep 6: Engineering features (minimal set)...")
    
    if 'time' not in df.columns:
        print("  ⚠ No 'time' column, skipping time features")
        return df
    
    df['time'] = pd.to_datetime(df['time'])
    
    # ONLY create the most important lag features
    # Lags: 1h, 6h, 24h (recent, medium-term, daily pattern)
    lag_hours = [1, 6, 24]
    
    if 'aqi' in df.columns:
        for lag in lag_hours:
            df[f'aqi_lag_{lag}h'] = df['aqi'].shift(lag)
        print(f"  ✓ Created {len(lag_hours)} AQI lag features")
    
    if 'pm2_5' in df.columns:
        for lag in lag_hours:
            df[f'pm25_lag_{lag}h'] = df['pm2_5'].shift(lag)
        print(f"  ✓ Created {len(lag_hours)} PM2.5 lag features")
    
    # ONLY create essential rolling features
    # 6h and 24h moving averages (short and daily patterns)
    if 'aqi' in df.columns:
        df['aqi_ma_6h'] = df['aqi'].rolling(window=6, min_periods=1).mean()
        df['aqi_ma_24h'] = df['aqi'].rolling(window=24, min_periods=1).mean()
        print("  ✓ Created 2 AQI rolling averages")
    
    # Cyclical time features (hour of day pattern)
    if 'hour' in df.columns:
        df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
        df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
        print("  ✓ Created cyclical hour features")
    
    # Day of week pattern (weekday vs weekend)
    if 'day_of_week' in df.columns:
        df['is_weekday'] = (df['day_of_week'] < 5).astype(int)
        print("  ✓ Created weekday indicator")
    
    # Simple PM ratio (if both available)
    if 'pm2_5' in df.columns and 'pm10' in df.columns:
        df['pm_ratio'] = df['pm2_5'] / (df['pm10'] + 0.1)  # Avoid division by zero
        print("  ✓ Created PM ratio")
    
    # Fill NaN from lag/rolling operations (backward then forward)
    df = df.fillna(method='bfill').fillna(method='ffill')
    
    print(f"✓ Total features: {len(df.columns)}")
    return df

# ============================================================================
# STEP 7: Final Validation
# ============================================================================

def validate_cleaned_data(df):
    """Final validation"""
    print("\nStep 7: Final validation...")
    
    # Check nulls
    null_count = df.isnull().sum().sum()
    if null_count > 0:
        print(f"  ⚠ {null_count} null values remain")
        for col in df.columns:
            nulls = df[col].isnull().sum()
            if nulls > 0:
                print(f"    {col}: {nulls}")
    else:
        print("  ✓ No null values")
    
    # Check duplicates
    dupes = df.duplicated().sum()
    if dupes > 0:
        df = df.drop_duplicates()
        print(f"  ✓ Removed {dupes} duplicates")
    else:
        print("  ✓ No duplicates")
    
    # Check infinite values
    inf_count = np.isinf(df.select_dtypes(include=[np.number])).sum().sum()
    if inf_count > 0:
        df = df.replace([np.inf, -np.inf], np.nan)
        df = df.fillna(method='ffill').fillna(method='bfill')
        print(f"  ✓ Replaced {inf_count} infinite values")
    else:
        print("  ✓ No infinite values")
    
    print(f"\n✓ Final dataset: {len(df)} rows × {len(df.columns)} columns")
    return df

# ============================================================================
# STEP 8: Save Data
# ============================================================================

def save_cleaned_data(df, output_file):
    """Save cleaned data"""
    print(f"\nStep 8: Saving to {output_file}")
    
    try:
        df.to_csv(output_file, index=False)
        file_size = os.path.getsize(output_file) / (1024 * 1024)
        print(f"✓ Saved {len(df)} rows")
        print(f"  File size: {file_size:.2f} MB")
        return True
    except Exception as e:
        print(f"✗ ERROR: {e}")
        return False

# ============================================================================
# MAIN PIPELINE
# ============================================================================

def main():
    """Run optimized cleaning pipeline"""
    
    print("\nStarting OPTIMIZED AQI Data Cleaning\n")
    
    # Load
    df = load_data(INPUT_FILE)
    if df is None:
        return False
    
    # Remove useless columns
    df = remove_useless_columns(df)
    
    # Clean missing values
    df = clean_missing_values_smart(df)
    
    # Handle outliers
    df = handle_outliers_smart(df)
    
    # Fix types
    df = fix_data_types(df)
    
    # Minimal feature engineering
    df = engineer_features_minimal(df)
    
    # Validate
    df = validate_cleaned_data(df)
    
    # Save
    success = save_cleaned_data(df, OUTPUT_FILE)
    
    # Generate report
    
    # Summary
    print("\n" + "="*70)
    print("CLEANING COMPLETE - OPTIMIZED FOR ML")
    print("="*70)
    print(f"\nInput:  {INPUT_FILE}")
    print(f"Output: {OUTPUT_FILE}")
    print(f"\nFinal: {len(df)} rows × {len(df.columns)} columns")
    
    # Show key improvements
    
    
    return True

if __name__ == "__main__":
    success = main()
    
    if success:
        print("\n✅ Data cleaning optimized for better ML accuracy!")
    else:
        print("\n❌ Pipeline failed")

AQI DATA CLEANING - OPTIMIZED VERSION

Starting OPTIMIZED AQI Data Cleaning


Step 1: Loading data from data/cleaned_aqi_data_v2.csv
✓ Loaded 4340 rows, 47 columns

Step 2: Removing useless columns...
✓ Removed 3 columns with 100% NaN: ['snwd', 'wpgt', 'tsun']
✓ Removed redundant columns: ['aqi_color', 'time_of_day']
✓ Columns: 47 → 42 (removed 5)

Step 3: Handling missing values (smart strategy)...
✓ Rows: 4340 → 4340 (lost 0)

Step 4: Handling outliers...
  pm10: Capped 17 extreme outliers
  carbon_monoxide: Capped 2 extreme outliers
  prcp: Capped 157 extreme outliers
  coco: Capped 62 extreme outliers
  aqi_lag_1h: Capped 62 extreme outliers
  aqi_lag_3h: Capped 62 extreme outliers
  aqi_lag_6h: Capped 62 extreme outliers
  aqi_lag_12h: Capped 61 extreme outliers
  aqi_lag_24h: Capped 61 extreme outliers
  aqi_ma_6h: Capped 1 extreme outliers
  aqi_std_6h: Capped 322 extreme outliers
  aqi_std_12h: Capped 524 extreme outliers
✓ Outliers handled

Step 5: Fixing data types...
  ✓ tim

In [2]:
df = pd.read_csv("data/cleaned_aqi_data_v2.csv")

In [3]:
df.columns


Index(['time', 'pm2_5', 'pm10', 'nitrogen_dioxide', 'ozone', 'temp', 'rhum',
       'prcp', 'wdir', 'wspd', 'pres', 'cldc', 'coco', 'aqi_pm25', 'aqi_pm10',
       'aqi_o3', 'aqi_no2', 'aqi', 'dominant_pollutant', 'aqi_category',
       'hour', 'day_of_week', 'day_of_month', 'month', 'is_weekend', 'season',
       'aqi_lag_1h', 'aqi_lag_3h', 'aqi_lag_6h', 'aqi_lag_12h', 'aqi_lag_24h',
       'pm25_lag_1h', 'pm25_lag_6h', 'pm25_lag_24h', 'aqi_ma_6h', 'aqi_std_6h',
       'aqi_ma_12h', 'aqi_std_12h', 'aqi_ma_24h', 'aqi_std_24h', 'pm25_ma_6h',
       'pm25_ma_24h', 'temp_rhum_interaction', 'pm_ratio', 'wind_pollution',
       'hour_sin', 'hour_cos', 'is_weekday'],
      dtype='object')

In [ ]:
Index(['time', 'pm2_5', 'pm10', 'nitrogen_dioxide', 'ozone', 'sulphur_dioxide',
       'carbon_monoxide', 'aqi_pm25', 'aqi_pm10', 'aqi', 'temp', 'rhum',
       'prcp', 'snwd', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun', 'cldc', 'coco',
       'hour', 'day_of_week', 'day_of_month', 'month', 'day_of_year',
       'is_weekend', 'time_of_day', 'season', 'aqi_lag_1h', 'aqi_lag_3h',
       'aqi_lag_6h', 'aqi_lag_12h', 'aqi_lag_24h', 'pm25_lag_1h',
       'pm25_lag_6h', 'pm25_lag_24h', 'aqi_ma_6h', 'aqi_std_6h', 'aqi_ma_12h',
       'aqi_std_12h', 'aqi_ma_24h', 'aqi_std_24h', 'pm25_ma_6h', 'pm25_ma_24h',
       'aqi_category', 'aqi_color'],
      dtype='object')